# Polarizability using automatic differentiation

Simple example for computing properties using (forward-mode)
automatic differentiation.
For a more classical approach and more details about computing polarizabilities,
see Polarizability by linear response.

In [1]:
using DFTK
using LinearAlgebra
using ForwardDiff

# Construct PlaneWaveBasis given a particular electric field strength
# Again we take the example of a Helium atom.
function make_basis(ε::T; a=10., Ecut=30) where {T}
    lattice=T(a) * I(3)  # lattice is a cube of $a$ Bohrs
    # Helium at the center of the box
    atoms     = [ElementPsp(:He, psp=load_psp("hgh/lda/He-q2"))]
    positions = [[1/2, 1/2, 1/2]]

    model = model_DFT(lattice, atoms, positions, [:lda_x, :lda_c_vwn];
                      extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                      symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])  # No k-point sampling on isolated system
end

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    @assert isdiag(basis.model.lattice)
    a  = basis.model.lattice[1, 1]
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    sum(rr .* ρ) * basis.dvol
end

# Function to compute the dipole for a given field strength
function compute_dipole(ε; tol=1e-8, kwargs...)
    scfres = self_consistent_field(make_basis(ε; kwargs...); tol)
    dipole(scfres.basis, scfres.ρ)
end;

With this in place we can compute the polarizability from finite differences
(just like in the previous example):

In [2]:
polarizability_fd = let
    ε = 0.01
    (compute_dipole(ε) - compute_dipole(0.0)) / ε
end

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -2.770882241757                   -0.52    9.0
  2   -2.772145754357       -2.90       -1.32    1.0
  3   -2.772170453846       -4.61       -2.51    1.0
  4   -2.772170673721       -6.66       -3.28    1.0
  5   -2.772170720962       -7.33       -3.86    2.0
  6   -2.772170722919       -8.71       -4.73    1.0
  7   -2.772170723013      -10.02       -5.35    2.0
  8   -2.772170723015      -11.81       -5.94    1.0
  9   -2.772170723015      -12.58       -6.48    2.0
 10   -2.772170723015      -13.66       -7.05    1.0
 11   -2.772170723015   +  -14.07       -8.30    2.0
n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -2.770786719809                   -0.52    9.0
  2   -2.772057034020       -2.90       -1.32    1.0
  3   -2.772083247825       -4.58       -2.52    1.0
  4   -2.772083385511       -6.86       -3.40 

1.7735579934269348

We do the same thing using automatic differentiation. Under the hood this uses
custom rules to implicitly differentiate through the self-consistent
field fixed-point problem.

In [3]:
polarizability = ForwardDiff.derivative(compute_dipole, 0.0)
println()
println("Polarizability via ForwardDiff:       $polarizability")
println("Polarizability via finite difference: $polarizability_fd")

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -2.770777165177                   -0.53    9.0
  2   -2.772056149135       -2.89       -1.31    1.0
  3   -2.772082966571       -4.57       -2.62    1.0
  4   -2.772083415736       -6.35       -4.08    2.0
  5   -2.772083417620       -8.72       -4.40    2.0
  6   -2.772083417803       -9.74       -5.32    1.0
  7   -2.772083417810      -11.11       -5.95    2.0
  8   -2.772083417811      -12.97       -6.57    1.0
  9   -2.772083417811      -13.71       -7.51    2.0
 10   -2.772083417811   +  -13.83       -7.69    2.0
 11   -2.772083417811      -14.07       -9.30    1.0

Polarizability via ForwardDiff:       1.7725349698559938
Polarizability via finite difference: 1.7735579934269348
